# 🎯 Treinamento YOLOv8 V2 - Detecção COM/SEM Adesivo
## Google Colab - GPU Acelerado - DUAL CLASS

**Importante:** Antes de começar, vá em `Runtime > Change runtime type` e selecione **GPU** como acelerador.

**Diferença da V1:** Este notebook treina o modelo para detectar **2 classes**:
- 🟢 **com_adesivo** (classe 0)
- 🟠 **sem_adesivo** (classe 1)

## 1️⃣ Instalação de Dependências

In [ ]:
# Instalar ultralytics (YOLOv8)
!pip install ultralytics -q

## 2️⃣ Verificar GPU

In [ ]:
import torch
from ultralytics import YOLO

print("="*60)
print("VERIFICAÇÃO DE AMBIENTE - GOOGLE COLAB V2")
print("="*60)

print(f"\n✓ PyTorch versão: {torch.__version__}")
print(f"✓ CUDA disponível: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✓ GPU detectada: {torch.cuda.get_device_name(0)}")
    print(f"✓ Memória GPU Total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    device = 0
else:
    print("⚠ GPU não detectada. Verifique se selecionou GPU em Runtime > Change runtime type")
    device = 'cpu'

## 3️⃣ Upload do Dataset

**Opção A:** Faça upload manual do dataset compactado (ZIP)

**IMPORTANTE:** Seu dataset deve ter imagens anotadas com **AMBAS** as classes:
- Labels com `0` (com_adesivo)
- Labels com `1` (sem_adesivo)

In [ ]:
from google.colab import files
import zipfile
import os

print("📦 Faça upload do arquivo ZIP contendo o dataset...")
uploaded = files.upload()

# Descompactar
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"\n📂 Descompactando {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print("✓ Dataset descompactado!")
        os.remove(filename)  # Remove o ZIP após descompactar

**Opção B:** Montar Google Drive (se o dataset estiver lá)

In [ ]:
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Copiar dataset do Drive (ajuste o caminho conforme necessário)
# !cp -r /content/drive/MyDrive/seu_dataset.zip .
# !unzip -q seu_dataset.zip

## 4️⃣ Criar arquivo data.yaml para 2 classes

In [ ]:
# Criar arquivo de configuração do dataset com 2 classes
data_yaml = """path: ./dataset
train: images/train
val: images/val

nc: 2
names: ['com_adesivo', 'sem_adesivo']
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml)

print("✓ Arquivo data.yaml criado com 2 classes!")
print("\nClasses configuradas:")
print("  0: com_adesivo")
print("  1: sem_adesivo")

## 5️⃣ Verificar estrutura do dataset e distribuição de classes

In [ ]:
import os

# Verificar se as pastas existem
print("📁 Verificando estrutura do dataset...\n")

paths_to_check = [
    'dataset/images/train',
    'dataset/images/val',
    'dataset/labels/train',
    'dataset/labels/val'
]

for path in paths_to_check:
    if os.path.exists(path):
        count = len(os.listdir(path))
        print(f"✓ {path}: {count} arquivos")
    else:
        print(f"⚠ {path}: NÃO ENCONTRADO")

# Verificar distribuição de classes nos labels
print("\n📊 Verificando distribuição de classes...\n")

def count_classes(label_dir):
    class_0_count = 0
    class_1_count = 0
    
    if os.path.exists(label_dir):
        for label_file in os.listdir(label_dir):
            if label_file.endswith('.txt'):
                with open(os.path.join(label_dir, label_file), 'r') as f:
                    for line in f:
                        class_id = int(line.split()[0])
                        if class_id == 0:
                            class_0_count += 1
                        elif class_id == 1:
                            class_1_count += 1
    
    return class_0_count, class_1_count

train_c0, train_c1 = count_classes('dataset/labels/train')
val_c0, val_c1 = count_classes('dataset/labels/val')

print("TREINO:")
print(f"  - com_adesivo (classe 0): {train_c0} anotações")
print(f"  - sem_adesivo (classe 1): {train_c1} anotações")

print("\nVALIDAÇÃO:")
print(f"  - com_adesivo (classe 0): {val_c0} anotações")
print(f"  - sem_adesivo (classe 1): {val_c1} anotações")

# Alerta se alguma classe está faltando
if train_c0 == 0 or train_c1 == 0:
    print("\n⚠️ ATENÇÃO: Uma ou mais classes não têm exemplos de treino!")
    print("O modelo não conseguirá aprender a detectar ambas as classes.")
    print("Verifique suas anotações no MakeSense.")
else:
    print("\n✅ Ambas as classes têm exemplos de treino!")
    total = train_c0 + train_c1
    print(f"Distribuição: com_adesivo {train_c0/total*100:.1f}% | sem_adesivo {train_c1/total*100:.1f}%")

## 6️⃣ TREINAMENTO V2 - DUAL CLASS 🚀

In [ ]:
print("="*60)
print("TREINAMENTO V2 - DETECÇÃO COM/SEM ADESIVO")
print("="*60)
print("\nClasses:")
print("  🟢 com_adesivo (classe 0)")
print("  🟠 sem_adesivo (classe 1)")
print("\nIniciando treinamento...")
print("="*60)

# Carregar modelo pré-treinado YOLOv8 nano
model = YOLO('yolov8n.pt')

# Treinar modelo com 2 classes
results = model.train(
    data='data.yaml',
    epochs=100,
    imgsz=640,
    batch=4,            # Ajuste conforme a GPU do Colab
    patience=20,
    device=device,
    project='adesivo_detection',
    name='v2_dual_class',  # Nome específico para versão V2
    save=True,
    plots=True,
    workers=2,
    verbose=True
)

print("\n" + "="*60)
print("✓ TREINAMENTO V2 CONCLUÍDO!")
print("="*60)
print(f"\n📊 Resultados salvos em: adesivo_detection/v2_dual_class/")
print(f"🏆 Melhor modelo: adesivo_detection/v2_dual_class/weights/best.pt")
print(f"📈 Gráficos: adesivo_detection/v2_dual_class/*.png")

## 7️⃣ Validação do Modelo - Métricas por Classe

In [ ]:
print("="*60)
print("Validando modelo V2...")
print("="*60)

# Carregar melhor modelo treinado
model = YOLO('adesivo_detection/v2_dual_class/weights/best.pt')
metrics = model.val()

print(f"\n📊 MÉTRICAS FINAIS (TODAS AS CLASSES):")
print(f"   mAP50: {metrics.box.map50:.3f}")
print(f"   mAP50-95: {metrics.box.map:.3f}")
print(f"   Precisão: {metrics.box.mp:.3f}")
print(f"   Recall: {metrics.box.mr:.3f}")

# Métricas por classe (se disponível)
if hasattr(metrics.box, 'maps'):
    print(f"\n📊 MÉTRICAS POR CLASSE:")
    class_names = ['com_adesivo', 'sem_adesivo']
    for i, name in enumerate(class_names):
        if i < len(metrics.box.maps):
            print(f"\n   {name.upper()}:")
            print(f"      mAP50-95: {metrics.box.maps[i]:.3f}")

print("\n✓ Validação concluída!")

## 8️⃣ Visualizar Gráficos de Treinamento

In [ ]:
from IPython.display import Image, display
import os

# Mostrar gráficos de resultados
results_path = 'adesivo_detection/v2_dual_class'

print("📊 GRÁFICOS DE TREINAMENTO V2:\n")

# Listar todos os gráficos disponíveis
plots = ['results.png', 'confusion_matrix.png', 'F1_curve.png', 'P_curve.png', 'R_curve.png', 'PR_curve.png']

for plot in plots:
    plot_path = os.path.join(results_path, plot)
    if os.path.exists(plot_path):
        print(f"\n{plot}:")
        display(Image(filename=plot_path))

## 9️⃣ Teste Rápido - Inferência em Uma Imagem

In [ ]:
from IPython.display import Image, display
import os

# Pegar uma imagem de validação aleatória
val_images = os.listdir('dataset/images/val')
if val_images:
    test_image = f"dataset/images/val/{val_images[0]}"
    
    print(f"🧪 Testando modelo em: {test_image}\n")
    
    # Fazer predição
    results = model.predict(source=test_image, conf=0.5, save=True)
    
    # Mostrar resultado
    print("\n📸 Resultado da detecção:")
    
    # Mostrar detecções encontradas
    for r in results:
        boxes = r.boxes
        if len(boxes) > 0:
            print(f"\n✓ Encontradas {len(boxes)} detecções:")
            for box in boxes:
                cls = int(box.cls)
                conf = float(box.conf)
                class_name = r.names[cls]
                print(f"  - {class_name}: {conf:.2%}")
        else:
            print("⚠ Nenhuma detecção encontrada")
    
    # Encontrar e mostrar a imagem salva
    pred_path = results[0].save_dir
    saved_img = os.path.join(pred_path, os.path.basename(test_image))
    if os.path.exists(saved_img):
        print(f"\n📷 Imagem com detecções:")
        display(Image(filename=saved_img))
else:
    print("⚠ Nenhuma imagem de validação encontrada para testar")

## 🔟 Download do Modelo Treinado V2

In [ ]:
from google.colab import files
import shutil

# Criar ZIP com os modelos e resultados
print("📦 Compactando resultados V2...")
shutil.make_archive('adesivo_detection_v2_results', 'zip', 'adesivo_detection/v2_dual_class')

print("⬇️ Fazendo download...")
files.download('adesivo_detection_v2_results.zip')

print("\n✓ Download concluído!")
print("\nO arquivo ZIP contém:")
print("  - weights/best.pt (melhor modelo V2 - 2 classes)")
print("  - weights/last.pt (último checkpoint)")
print("  - Todos os gráficos e métricas")
print("\n📋 Próximos passos:")
print("  1. Extraia o ZIP")
print("  2. Copie best.pt para seu projeto local")
print("  3. Execute testv2.py para testar com máscaras coloridas!")

## 1️⃣1️⃣ (Opcional) Salvar no Google Drive

In [ ]:
from google.colab import drive

# Montar Drive se ainda não estiver montado
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Copiar resultados para o Google Drive
!cp -r adesivo_detection/v2_dual_class /content/drive/MyDrive/adesivo_detection_v2_results

print("✓ Resultados V2 salvos no Google Drive!")
print("📁 Localização: /MyDrive/adesivo_detection_v2_results")

---

## 📝 Notas Importantes - VERSÃO V2:

### **Diferenças da V1:**

| Aspecto | V1 (train_colab.ipynb) | V2 (train_colab_v2.ipynb) |
|---------|------------------------|---------------------------|
| Classes | 1 (só com_adesivo) | 2 (com/sem adesivo) |
| Pasta de saída | run1 | v2_dual_class |
| Teste | test.py | testv2.py |
| Máscaras | Verde apenas | 🟢 Verde + 🟠 Laranja |

### **Requisitos do Dataset V2:**

✅ **OBRIGATÓRIO:** Seu dataset deve ter anotações das 2 classes:

- Labels com `0` no início da linha = com_adesivo
- Labels com `1` no início da linha = sem_adesivo

**Exemplo de label válido:**
```
0 0.5 0.3 0.2 0.4  ← com_adesivo
1 0.7 0.6 0.15 0.3 ← sem_adesivo
```

### **Uso do Modelo Treinado:**

1. **Download:** Baixe o `adesivo_detection_v2_results.zip`
2. **Extrair:** Extraia o arquivo ZIP
3. **Copiar:** Coloque `best.pt` em `adesivo_detection/v2_dual_class/weights/best.pt`
4. **Testar:** Execute `python testv2.py` no seu computador local

### **Ajustes de Performance:**

- **Erro de memória:** Reduza `batch=4` para `batch=2`
- **Treinamento lento:** Verifique se GPU está ativa
- **Métricas baixas:** Adicione mais imagens anotadas de ambas as classes

### **Dicas:**

- ✅ Mantenha proporção similar entre as classes (50/50 ideal)
- ✅ Mínimo recomendado: 50 imagens de cada classe
- ✅ Anote todas as ocorrências em cada imagem
- ✅ Valide no MakeSense antes de exportar

---

## 🚀 Após o Download:

Execute no seu computador local:

```bash
# Testar o modelo V2 com máscaras coloridas
python testv2.py
```

O testv2.py vai mostrar:
- 🟢 **Máscara verde** para COM_ADESIVO
- 🟠 **Máscara laranja** para SEM_ADESIVO
- Contadores separados por classe
- Legenda de cores no rodapé